In [1]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import cv2
import numpy as np
import pandas as pd
import os
import pickle
import random
import time
import importlib

import CNN_encoder as cn
import functions as fc

# 讀取圖片位置

In [2]:
train_df = pd.read_csv('./dataset/train.txt', sep=' ', header=None)
test_df = pd.read_csv('./dataset/test.txt', sep=' ', header=None)
val_df = pd.read_csv('./dataset/val.txt', sep=' ', header=None)
train_paths, train_response = './dataset/' + train_df[0].to_numpy(), train_df[1].to_numpy()
test_paths, test_response = './dataset/' + test_df[0].to_numpy(), test_df[1].to_numpy()
val_paths, val_response = './dataset/' + val_df[0].to_numpy(), val_df[1].to_numpy()
train_paths.shape, test_paths.shape, val_paths.shape

((63325,), (450,), (450,))

# 使用CNN做特徵提取

In [3]:
importlib.reload(fc)
importlib.reload(cn)

# 訓練用超參數
batch_size = 20
epochs = 100
lr = 0.01
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

logger = fc.create_logger('./logger/', 'encoder_logger.txt')
model = cn.CNN_model_encoder().to(device)
lossf = nn.MSELoss().to(device)
opt = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
print(device)

cpu


# train encoder

In [4]:
# load model
# model = cn.CNN_model_encoder().to(device)
# model.load_state_dict(torch.load('./logger/CNN_model_encoder.pt'))

start = time.time()
logger.info('start at {}'.format(start))

for epoch in range(1, epochs+1):
    img_index = 0
    img_len = train_paths.shape[0]
    while img_index < img_len:
        try:
            imgs = train_paths[img_index:img_index + batch_size]
            pic = fc.path2pic(imgs)
            inputs = fc.pic2tensor(pic).to(device)
            outputs = model(inputs)
            loss1 = lossf(outputs[0], outputs[9])
            loss2 = lossf(outputs[1], outputs[8])
            loss3 = lossf(outputs[2], outputs[7])
            loss4 = lossf(outputs[3], outputs[6])
            loss5 = lossf(outputs[4], outputs[5])
            loss = (loss1 + loss2 + loss3 + loss4 + loss5) / 5
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            img_index += batch_size
            torch.save(model.state_dict(), './model/CNN_model_encoder.pt')
            
        except Exception as e:
            print(img_index)
            logger.info(e, exc_info=True)
    
    train_time = time.time() - start
    torch.save(model.state_dict(), './model/CNN_model_encoder.pt')
    logger.info('train time {}, {}/{}'.format(train_time, epoch, epochs))

2022-08-11 15:16:11,102 INFO start at 1660202171.1022706
2022-08-11 15:16:12,869 INFO train time 1.7543730735778809, 1/100


# feature extract

# for train data

# 讀取圖片位置

In [4]:
train_df = pd.read_csv('./dataset/train.txt', sep=' ', header=None)
test_df = pd.read_csv('./dataset/test.txt', sep=' ', header=None)
val_df = pd.read_csv('./dataset/val.txt', sep=' ', header=None)
train_paths, train_response = './dataset/' + train_df[0].to_numpy(), train_df[1].to_numpy()
test_paths, test_response = './dataset/' + test_df[0].to_numpy(), test_df[1].to_numpy()
val_paths, val_response = './dataset/' + val_df[0].to_numpy(), val_df[1].to_numpy()
train_paths.shape, test_paths.shape, val_paths.shape

((63325,), (450,), (450,))

In [5]:
# load model
importlib.reload(fc)
importlib.reload(cn)

# 訓練用超參數
batch_size = 20
epochs = 100
lr = 0.01
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

logger = fc.create_logger('./logger/', 'encoder_logger.txt')
model = cn.CNN_model_encoder().to(device)
model.load_state_dict(torch.load('./model/CNN_model_encoder.pt'))
print(device)

<All keys matched successfully>

In [6]:
batch_size = 20
img_index = 0
img_len = val_paths.shape[0]
features = []

while img_index < img_len:
    imgs = val_paths[img_index:img_index + batch_size]
    pic = fc.path2pic(imgs)
    inputs = fc.pic2tensor(pic).to(device)
    outputs = model.feature_extract(inputs)
    img_index += batch_size
    features.append(outputs)
    logger.info('img_index:{}'.format(img_index))
feature_array = torch.cat(features, dim=0)
feature_array_cpu = feature_array.clone().detach().cpu()
torch.save(feature_array_cpu, './imgFeature/valFeature.pt')
del(feature_array)
del(feature_array_cpu)

2021-03-28 14:48:43,432 INFO img_index:20
2021-03-28 14:48:45,217 INFO img_index:40
2021-03-28 14:48:46,744 INFO img_index:60
2021-03-28 14:48:48,383 INFO img_index:80
2021-03-28 14:48:49,968 INFO img_index:100
2021-03-28 14:48:51,656 INFO img_index:120
2021-03-28 14:48:53,305 INFO img_index:140
2021-03-28 14:48:54,893 INFO img_index:160
2021-03-28 14:48:56,410 INFO img_index:180
2021-03-28 14:48:57,928 INFO img_index:200
2021-03-28 14:48:59,448 INFO img_index:220
2021-03-28 14:49:00,968 INFO img_index:240
2021-03-28 14:49:02,444 INFO img_index:260
2021-03-28 14:49:03,977 INFO img_index:280
2021-03-28 14:49:05,539 INFO img_index:300
2021-03-28 14:49:07,053 INFO img_index:320
2021-03-28 14:49:08,566 INFO img_index:340
2021-03-28 14:49:10,099 INFO img_index:360
2021-03-28 14:49:11,612 INFO img_index:380
2021-03-28 14:49:13,143 INFO img_index:400
2021-03-28 14:49:14,612 INFO img_index:420
2021-03-28 14:49:16,164 INFO img_index:440
2021-03-28 14:49:16,908 INFO img_index:460


# try

In [3]:
path_norm = './train/NORMAL/'
imglist_norm = os.listdir(path_norm)
path_pneu = './train/PNEUMONIA/'
imglist_pneu = os.listdir(path_pneu)
img_list = []
for i in imglist_norm:
    img_list.append(path_norm + i)
    
for i in imglist_pneu:
    img_list.append(path_pneu + i)
    
random.shuffle(img_list)
len(img_list)

5216

In [4]:
# 訓練用參數
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
model = CNN_model_encoder().to(device)
lossf = nn.MSELoss()

In [6]:
imgs = img_list[0:10]
pic = path2pic(imgs)
inputs = pic2tensor(pic).to(device)

In [7]:
inputs.shape

torch.Size([10, 3, 512, 512])

In [8]:
outputs = model(inputs)

In [11]:
loss5 = lossf(outputs[0].detach(), outputs[14])

In [15]:
loss5.backward(retain_graph=True)

In [21]:
loss1 = lossf(outputs[1].detach(), outputs[6])
loss2 = lossf(outputs[2].detach(), outputs[5])
loss3 = lossf(outputs[8].detach(), outputs[13])
loss4 = lossf(outputs[9].detach(), outputs[12])
loss5 = lossf(outputs[0].detach(), outputs[14])

In [22]:
loss1, loss2, loss3, loss4, loss5

(tensor(0.6279, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(1.3595, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(1.4133, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(1.4666, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(0.8438, device='cuda:0', grad_fn=<MseLossBackward>))

In [23]:
loss1.backward(retain_graph=True)

In [24]:
loss2.backward(retain_graph=True)

In [48]:
# With square kernels and equal stride
m = nn.ConvTranspose2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
# m = nn.ConvTranspose2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
input = torch.randn(20, 16, 50, 100)
output = m(input)
# exact output size can be also specified as an argument
input = torch.randn(1, 16, 12, 12)
downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1)
upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1)
h = downsample(input)
h.size()
torch.Size([1, 16, 6, 6])
output = upsample(h, output_size=input.size())
output.size()
torch.Size([1, 16, 12, 12])

torch.Size([1, 16, 12, 12])

In [52]:
# With square kernels and equal stride
m = nn.ConvTranspose2d(16, 33, 3, stride=1)
input = torch.randn(20, 16, 50, 100)

In [53]:
output = m(input)
output.shape

torch.Size([20, 33, 52, 102])

In [65]:
m = nn.Conv2d(3, 1, (3, 3), stride=1)

In [61]:
len(list(m.parameters()))

2

In [62]:
list(m.parameters())[0].shape, list(m.parameters())[1].shape

(torch.Size([8, 3, 3, 3]), torch.Size([8]))

In [66]:
input = torch.randn(20, 3, 50, 100)

In [68]:
m(input).shape

torch.Size([20, 1, 48, 98])

In [94]:
input = torch.rand(2, 1, 5, 5)
input, input.shape

(tensor([[[[0.3824, 0.2618, 0.8074, 0.4320, 0.8458],
           [0.4926, 0.1995, 0.1289, 0.6962, 0.2754],
           [0.2103, 0.5003, 0.0784, 0.1984, 0.2855],
           [0.9783, 0.1403, 0.6125, 0.6148, 0.9563],
           [0.8009, 0.4543, 0.0335, 0.1047, 0.0306]]],
 
 
         [[[0.2588, 0.2052, 0.7625, 0.1607, 0.3166],
           [0.3952, 0.2438, 0.9578, 0.8956, 0.9670],
           [0.6519, 0.7244, 0.5740, 0.9927, 0.9858],
           [0.4974, 0.8008, 0.7410, 0.8023, 0.8763],
           [0.3673, 0.3164, 0.7160, 0.3681, 0.2028]]]]),
 torch.Size([2, 1, 5, 5]))

In [95]:
m = nn.AdaptiveMaxPool2d((4, 4))
output = m(input)
output, output.shape

(tensor([[[[0.4926, 0.8074, 0.8074, 0.8458],
           [0.5003, 0.5003, 0.6962, 0.6962],
           [0.9783, 0.6125, 0.6148, 0.9563],
           [0.9783, 0.6125, 0.6148, 0.9563]]],
 
 
         [[[0.3952, 0.9578, 0.9578, 0.9670],
           [0.7244, 0.9578, 0.9927, 0.9927],
           [0.8008, 0.8008, 0.9927, 0.9927],
           [0.8008, 0.8008, 0.8023, 0.8763]]]]),
 torch.Size([2, 1, 4, 4]))

In [3]:
x = torch.rand(5, 3, 10, 10)

In [12]:
m = nn.LayerNorm(x.shape[2:], elementwise_affine=False)

In [13]:
output = m(x)

In [14]:
output

tensor([[[[ 0.5829,  1.4797,  0.4275,  ...,  0.0336, -0.5549, -0.4246],
          [ 0.1870, -0.1482,  0.5749,  ..., -1.0905, -1.6453,  0.7862],
          [ 0.6661, -0.2262, -1.0174,  ..., -1.4119,  1.2582, -0.2301],
          ...,
          [ 1.3639,  0.3099,  0.6002,  ..., -0.6457,  1.5809,  1.2038],
          [ 0.3407,  1.5708, -1.5265,  ...,  0.3097, -1.5203, -0.2271],
          [ 0.0590, -1.6453, -0.5070,  ...,  0.0652, -0.1709, -0.0558]],

         [[ 0.7759, -1.2816,  1.4900,  ...,  0.8405,  0.5381, -0.0728],
          [ 0.2517,  0.4695, -1.1645,  ...,  0.2120,  0.8396, -1.4304],
          [-0.7659, -0.2544,  0.8152,  ..., -0.2501,  0.8558, -0.1098],
          ...,
          [-0.3252,  1.3697, -0.7355,  ..., -1.3774,  1.4510, -0.4577],
          [ 1.3610, -0.1968, -0.1480,  ...,  0.0546, -0.2145,  1.2643],
          [-0.8530, -0.9829,  0.7633,  ...,  1.2993, -1.8078,  1.3453]],

         [[-1.3533, -0.2200,  1.4388,  ..., -0.4057, -1.2009,  0.2431],
          [ 0.5953, -0.6241,  